# Курсовая работа
## 10 вариант: Вычисление несобственных интегралов численными методами

Определённый интеграл называется собственным, если выполняется по крайней мере одно из следующих условий:

1. Область интегрирования является бесконечной - интеграл 1 рода.

2. Подынтегральная функция является неограниченной в окрестности некоторых точек области интегрирования - интеграл 2 рода.

Несобственный интеграл 2 рода можно свести к интегралу 1 рода с помощью замены переменной. Поэтому будем рассматривать только несобственные интегралы 1 рода.

### 1. Сведение к определённому интегралу

$\ \ $Рассмотрим преобразование из математического анализа, выполненное с помощью замены переменной:

$
\int_a^b f(x)dx = \int_{\frac{1}{a}}^{\frac{1}{b}} \frac{1}{t^2}f(\frac{1}{t})dt \quad при \ ab > 0
$<br>

$\ \ $Можем разложить несобственный интеграл на сумму интегралов.

$
\int_{-∞}^{+∞} f(x)dx = \int_{-∞}^{-A} f(x)dx + \int_{-A}^{B} f(x)dx + \int_{B}^{+∞} f(x)dx \quad при \ {-A} < 0 \ и \ B > 0
$<br>

$\ \ $Первый и последний интегралы можем преобразовать с помощью формулы выше. Так мы можем посчитать каждый из этих трёх интегралов (например, методов прямоугольников) и сложить получившиеся результаты.

### 2. Предельный переход

$\ \ $Запишем предельный переход для несобственного интеграла 1 рода:

$
\int_a^{+∞} f(x)dx = \lim_{b \to \infty} \int_a^b f(x)dx
$

$\ \ $Будем вычислять правый интеграл (например, методом прямоугольников) до тех пор, пока следующее слагаемое не станет меньше заданного эпсилон

In [75]:
import math

In [76]:
INF = 1e10

In [77]:
# Функция интегрирования
def f(x):
    return 1 / (1 + x**2)

In [78]:
# Высчитываем интеграл f(x)dx на интервале [l; r] используя метод левых прямоугольников с шагом=h
def integrate_rectangle_left_method(f, l, r, h):
    n = int((r - l) / h)
    s = 0
    for i in range(n):
        s += h * f(l + h * i)
    return s

In [79]:
# Высчитываем интеграл f(x)dx на интервале [l; r] используя метод правых прямоугольников с шагом=h
def integrate_rectangle_right_method(f, l, r, h):
    n = int((r - l) / h)
    s = 0
    for i in range(n):
        s += h * f(l + h * (i + 1))
    return s

In [80]:
# Высчитываем интеграл f(x)dx на интервале [l; r] используя метод центральных прямоугольников с шагом=h
def integrate_rectangle_method(f, l, r, h):
    n = int((r - l) / h)
    s = 0
    for i in range(n):
        s += h * f(l + h * (i + 0.5))

    return s

In [81]:
# Высчитываем интеграл f(x)dx на интервале [l; r] используя метод трапеций с шагом=h
def integrate_trapezoid_method(f, l, r, h):
    n = int((r - l) / h)
    x = [l + i * h for i in range(n + 1)]
    s = 0
    for i in range(n):
        s += h * (f(x[i]) + f(x[i + 1])) / 2
    return s

In [82]:
# Высчитываем интеграл f(x)dx на интервале [l; r] используя метод парабол (формула Симпсона) с шагом=h
def integrate_parabola_method(f, l, r, h):
    n = int((r - l) / (h / 2) )

    x = [l + i * (h / 2) for i in range(1, n)]

    s1, s2 = 0, 0
    for i in range(len(x)):
        if i % 2:
            s1 += f(x[i])
        else:
            s2 += f(x[i])

    return (h / 2) / 3 * (f(l) + 2 * s1 + 4 * s2 + f(r))

In [83]:
# Вычисляем несобственный интеграл (1 типа), преобразуя его в определённый интеграл
def integrate_with_definite_integral(integration_method, f, l, r, h=0.01, eps=1e-6):
    def f_new(t):
        return (1. / t ** 2) * f(1. / t)

    result = 0
    if r == INF:
        new_r = max(eps, l)
        result += integration_method(f_new, eps, 1. / new_r - eps, h)
    else:
        new_r = r
    if l == -INF:
        new_l = min(-eps, r)
        result += integration_method(f_new, 1. / new_l + eps, -eps, h)
    else:
        new_l = l
    if new_l < new_r:
        result += integration_method(f, new_l, new_r, h)
    return result

In [84]:
# Вычисляем несобственный интеграл f(x)dx (1 типа), используя предельный переход. Возвращаем: результат интегрирования и количество итераций
def integrate_lim(integration_method, f, l, r, h=0.1, eps=1e-6):
    result = 0
    iters = 0
    if r == INF:
        finish = False
        cur_x = max(l, 0)
        while not finish:
            iters += 1
            diff = integration_method(f, cur_x, cur_x + h + eps, h)
            cur_x += h
            if abs(diff) < eps:
                finish = True
            result += diff
        result += f(cur_x - h) * (f(cur_x - h) * h / (f(cur_x - h) - f(cur_x))) # Правый треугольник
    else:
        result += integration_method(f, 0, r, h)
    if l == -INF:
        finish = False
        cur_x = min(0, r)
        while not finish:
            iters += 1
            diff = integration_method(f, cur_x - h - eps, cur_x, h)
            cur_x -= h
            if abs(diff) < eps:
                finish = True
            result += diff
        result += f(cur_x + h) * (f(cur_x + h) * h / (f(cur_x + h) - f(cur_x))) # Левый треугольник
    else:
        result += integration_method(f, l, 0, h)
    return result, iters

In [85]:
def CP(a, b, h, eps):
    print('Метод центральных прямоугольников:')
    print('Преобразование в определённый интеграл')
    res_definite = integrate_with_definite_integral(integrate_rectangle_method, f, a, b, h, eps)
    print('Интеграл =', res_definite)
    print()
    print('Метод предельного перехода')
    res_limit, iters_limit = integrate_lim(integrate_rectangle_method, f, a, b, h, eps)
    print('Интеграл =', res_limit)
    print('Количество итераций:', iters_limit)

    print()
    print('------------------------------------------------------------------------------------')
    print()

    print('Метод трапеций:')
    print('Преобразование в определённый интеграл')
    res_definite = integrate_with_definite_integral(integrate_trapezoid_method, f, a, b, h, eps)
    print('Интеграл =', res_definite)
    print()
    print('Метод предельного перехода')
    res_limit, iters_limit = integrate_lim(integrate_trapezoid_method, f, a, b, h, eps)
    print('Интеграл =', res_limit)
    print('Количество итераций:', iters_limit)

    print()
    print('------------------------------------------------------------------------------------')
    print()

    print('Метод парабол:')
    print('Преобразование в определённый интеграл')
    res_definite = integrate_with_definite_integral(integrate_parabola_method, f, a, b, h, eps)
    print('Интеграл =', res_definite)
    print()
    print('Метод предельного перехода')
    res_limit, iters_limit = integrate_lim(integrate_parabola_method, f, a, b, h, eps)
    print('Интеграл =', res_limit)
    print('Количество итераций:', iters_limit)

## Результат работы программы

Будем вычислять следующий интеграл: $\int_l^r \frac{1}{1 + x^2}$

In [86]:
l = 3
r = INF
h = 0.01
eps = 1e-6

CP(l, r, h, eps)

Метод центральных прямоугольников:
Преобразование в определённый интеграл
Интеграл = 0.3187496986594376

Метод предельного перехода
Интеграл = 0.3167518878315057
Количество итераций: 9700

------------------------------------------------------------------------------------

Метод трапеций:
Преобразование в определённый интеграл
Интеграл = 0.3187429893745884

Метод предельного перехода
Интеграл = 0.3167531374050679
Количество итераций: 9701

------------------------------------------------------------------------------------

Метод парабол:
Преобразование в определённый интеграл
Интеграл = 0.31874447307371695

Метод предельного перехода
Интеграл = 0.31858545455593584
Количество итераций: 9700


In [87]:
l = -INF
r = -9
h = 0.01
eps = 1e-8

CP(l, r, h, eps)

Метод центральных прямоугольников:
Преобразование в определённый интеграл
Интеграл = 0.1095470048929639

Метод предельного перехода
Интеграл = 0.11015722773111694
Количество итераций: 99101

------------------------------------------------------------------------------------

Метод трапеций:
Преобразование в определённый интеграл
Интеграл = 0.10954432216582874

Метод предельного перехода
Интеграл = 0.1101572611933085
Количество итераций: 99101

------------------------------------------------------------------------------------

Метод парабол:
Преобразование в определённый интеграл
Интеграл = 0.10954611270815913

Метод предельного перехода
Интеграл = 0.11184423077544711
Количество итераций: 99101


In [88]:
l = -INF
r = 10
h = 0.01
eps = 1e-4

CP(l, r, h, eps)

Метод центральных прямоугольников:
Преобразование в определённый интеграл
Интеграл = 3.0320232609183275

Метод предельного перехода
Интеграл = 2.9920847624718414
Количество итераций: 996

------------------------------------------------------------------------------------

Метод трапеций:
Преобразование в определённый интеграл
Интеграл = 3.0320234813678235

Метод предельного перехода
Интеграл = 2.992084715665344
Количество итераций: 996

------------------------------------------------------------------------------------

Метод парабол:
Преобразование в определённый интеграл
Интеграл = 3.035356504057397

Метод предельного перехода
Интеграл = 2.992101246771739
Количество итераций: 996


In [89]:
l = -INF
r = INF
h = 0.01
eps = 1e-5

CP(l, r, h, eps)

Метод центральных прямоугольников:
Преобразование в определённый интеграл
Интеграл = 3.131572902603841

Метод предельного перехода
Интеграл = 3.1100031831645705
Количество итераций: 6324

------------------------------------------------------------------------------------

Метод трапеций:
Преобразование в определённый интеграл
Интеграл = 3.1315731525602404

Метод предельного перехода
Интеграл = 3.1100031818361846
Количество итераций: 6324

------------------------------------------------------------------------------------

Метод парабол:
Преобразование в определённый интеграл
Интеграл = 3.1382394862722656

Метод предельного перехода
Интеграл = 3.1100031827208987
Количество итераций: 6324
